In [10]:
!pip install sagemaker -q

In [11]:
import tarfile
import sagemaker
from sagemaker.tensorflow.serving import Model
from tensorflow.keras.applications.resnet50 import ResNet50

In [ ]:
def load_save_resnet50_model(saved_model_dir = 'resnet50_saved_model'):
    model = ResNet50(weights='imagenet')
    shutil.rmtree(saved_model_dir, ignore_errors=True)
    model.save(saved_model_dir, include_optimizer=False, save_format='tf')

In [7]:
!tar cvfz model.tar.gz -C resnet50_saved_model .

./
./variables/
./variables/variables.data-00001-of-00002
./variables/variables.data-00000-of-00002
./variables/variables.index
./saved_model.pb
./assets/


In [13]:
# role = sagemaker.get_execution_role()
role = 'AmazonSageMaker-ExecutionRole-20190820T113591'
sess = sagemaker.Session()
region = sess.boto_region_name
bucket = sess.default_bucket()

prefix = 'keras_models'
model_path = sess.upload_data(path='model.tar.gz', key_prefix=prefix)

model = Model(model_data=model_file, 
              framework_version='2.2',
              role=role)

ValueError: Must setup local AWS configuration with a region supported by SageMaker.

In [ ]:
data_shape = '{"input0":[1,3,224,224]}'
target_device = 'ml_c5'
framework = 'pytorch'
framework_version = '1.4.0'
compiled_model_path = 's3://{}/{}/output'.format(bucket, compilation_job_name)

inference_image_uri = image_uris.retrieve(f'neo-{framework}', region, framework_version, instance_type=target_device)